In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(words[:8])
print(len(words))

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocab of characters and mapping to and from the integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{0: '.', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


In [4]:
# building the dataset

block_size = 3
X, Y = [], []

for w in words[:5]:

    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]  # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
olivia
ava
isabella
sophia


In [5]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [6]:
C = torch.randn((27, 2))  # embedding layer

In [10]:
emb = C[X]
print(emb.shape)

torch.Size([32, 3, 2])


In [7]:
# F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([0.7369, 1.4456])

In [11]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [12]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
print(h.shape)

torch.Size([32, 100])


In [13]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [14]:
logits = h @ W2 + b2
print(logits.shape)

torch.Size([32, 27])


In [16]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
print('Every row of prob sums to', prob[0].sum())

Every row of prob sums to tensor(1.0000)


In [20]:
loss = -prob[torch.arange(32), Y].log().mean()
print(loss)

tensor(12.7919)


### Now all at once, properly

In [26]:
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [31]:
g = torch.Generator().manual_seed(42)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [32]:
print('Total number of parameters:', sum(p.nelement() for p in parameters))

Total number of parameters: 3481


In [33]:
emb = C[X]  # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2  # (32, 27)
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(32), Y].log().mean()
print('Loss:', loss)

Loss: tensor(22.0323)
